In [ ]:
from deuces import *
from th_poker_ranking import rank_array
import itertools

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
C = Card()
E = Evaluator()
def rank_deuces_hand(hand):
    # Add two to the numbers since deuces does numbering for 0-12 and not 2-14. Wheel is not correct.
    return rank_array([(C.get_rank_int(card)+2, C.get_suit_int(card)) for card in hand], canadian=True)

# Evaluate single card

In [ ]:
n_players = 2
card_wins = {}
card_count = {}
for c in Deck().draw(52):
    card_wins[c] = 0
    card_count[c] = 0

for i in range(10000):
    deck = Deck()
    hands = [deck.draw(5) for i in range(n_players)]
    scores = [rank_deuces_hand(hand) for hand in hands]
    winner = scores.index(max(scores))
    for hand_index, hand in enumerate(hands):
        for card in hand:
            card_count[card] += 1
            if hand_index == winner:
                card_wins[card] += 1

In [ ]:
import pandas as pd 
df = pd.DataFrame()
df['card'] = card_count.keys()
df['card_wins'] = card_wins.values()
df['card_count'] = card_count.values()
df['equity'] = df['card_wins']/df['card_count']
df['pretty_str'] = df.card.apply(lambda x: C.int_to_pretty_str(x))
df.sort_values('equity', ascending=False).head(5)

# Evaluate visible cards

In [ ]:
def add_one(dictionary, key):
    if key not in dictionary:
        dictionary[key] = 0
    dictionary[key] += 1

def val(c):
    return C.get_rank_int(c)+2
    
n_players = 2
card_wins = {}
card_counts = {}

for i in range(200000):
    deck = Deck()
    hands = [deck.draw(5) for i in range(n_players)]
    scores = [rank_deuces_hand(hand) for hand in hands]
    winner = scores.index(max(scores))
    for c1 in hands[0]:
        for c2 in hands[1]:
            key = (val(c1), val(c2))
            add_one(card_counts, key)
            if 0 == winner:
                add_one(card_wins, key)

In [ ]:
df = pd.DataFrame()
df['count'] = card_counts.values()
df.index = card_counts.keys()

df_wins = pd.DataFrame()
df_wins['wins'] = card_wins.values()
df_wins.index = card_wins.keys()

df['wins'] = df_wins['wins']

df['equity'] = df['wins']/df['count']
df = df.reset_index()
df['my_visible_card'] = df['index'].apply(lambda x: x[0])
df['opponent_visible_card'] = df['index'].apply(lambda x: x[1])
# df['pretty_str'] = df.card.apply(lambda x: C.int_to_pretty_str(x))
pivot = pd.pivot(index=df['my_visible_card'], columns=df['opponent_visible_card'], values=df['equity'])

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
plt.imshow(pivot)

In [ ]:
df['diff'] = df['my_visible_card']-df['opponent_visible_card']

In [ ]:
plt.scatter(df['diff'], df['equity'])
df['aces_me'] = df['my_visible_card'] == 14
plt.scatter(df[df['aces_me']]['diff'], df[df['aces_me']]['equity'])
df['aces_opponent'] = df['opponent_visible_card'] == 14
plt.scatter(df[df['aces_opponent']]['diff'], df[df['aces_opponent']]['equity'])

# import seaborn as sns
# plt.scatter(x='diff', y='equity', hue="my_visible_card", data=df)

In [ ]:
from sklearn import linear_model

In [ ]:
model = linear_model.LinearRegression()
model.fit(df[['diff', 'aces_me', 'aces_opponent']], df['equity'])

In [ ]:
model.coef_

# Evaluate hideen cards + visible cards

In [ ]:
def add_one(dictionary, key):
    if key not in dictionary:
        dictionary[key] = 0
    dictionary[key] += 1

def val(c):
    return C.get_rank_int(c)+2
    
n_players = 2
card_wins = {}
card_counts = {}

for i in range(200000):
    deck = Deck()
    hands = [deck.draw(5) for i in range(n_players)]
    scores = [rank_deuces_hand(hand) for hand in hands]
    winner = scores.index(max(scores))
    for m1, m2 in itertools.combinations(hands[0], 2):
        my_cards_suited = C.get_suit_int(m1) == C.get_suit_int(m2)
        for c2 in hands[1]:
            key = (val(m2), val(m1), my_cards_suited, val(c2))
            add_one(card_counts, key)
            if 0 == winner:
                add_one(card_wins, key)

In [ ]:
df = pd.DataFrame()
df['count'] = card_counts.values()
df.index = card_counts.keys()

df_wins = pd.DataFrame()
df_wins['wins'] = card_wins.values()
df_wins.index = card_wins.keys()

df['wins'] = df_wins['wins']

df['equity'] = df['wins']/df['count']
df = df.reset_index()
df['my_hidden_card'] = df['index'].apply(lambda x: x[0])
df['my_visible_card'] = df['index'].apply(lambda x: x[1])
df['my_cards_suited'] = df['index'].apply(lambda x: x[2])
df['opponent_visible_card'] = df['index'].apply(lambda x: x[3])
# df['pretty_str'] = df.card.apply(lambda x: C.int_to_pretty_str(x))
# pivot = pd.pivot(index=df['my_visible_card'], columns=df[['opponent_visible_card']], values=df['equity'])

In [ ]:
mask = (
    (df['my_visible_card'] == 10)
    & (df['opponent_visible_card'] == 5)
)
df[mask & df['my_cards_suited']].sort_values('my_hidden_card').plot.bar('my_hidden_card', 'equity')
df[mask & (df['my_cards_suited'] == False)].sort_values('my_hidden_card').plot.bar('my_hidden_card', 'equity')

# Evaluate current position

In [ ]:
from copy import deepcopy
from joblib import Parallel, delayed

In [ ]:
def run_simulation(hands):
    MC_hands = deepcopy(hands)
    deck = Deck()
    for hand in hands:
        for card in hand:
            deck.remove(card)
        MC_hands = deepcopy(hands)

    for hand in MC_hands:
        while len(hand) < 5:
            hand.append(deck.draw())

    scores = [rank_deuces_hand(hand) for hand in MC_hands]
    best_score = max(scores)
    if scores[0] == best_score:
        return 1 / scores.count(best_score)
    return 0

In [ ]:
original_hands = [[], []]
N = 100
equity = {}
for hidden_1 in Deck.GetFullDeck():
    cards_left = Deck()
    cards_left.remove(hidden_1)
    for hidden_2 in cards_left.cards:
        hands = deepcopy(original_hands)
        hands[0].append(hidden_1)
        hands[1].append(hidden_2)
        
        if False:
            wins = [run_simulation(hands) for i in range(N)]
        else:
            if __name__ == '__main__':
                wins = Parallel(3)(delayed(run_simulation)(hands) for i in range(N))
        equity[(hidden_1, hidden_2)] = sum(wins)/N
        
plt.hist(list(equity.values()))

In [ ]:
def hand_string(hand):
    return " ".join([C.int_to_pretty_str(card) for card in hand])

In [ ]:
N = 1000

hands = [[], []]
deck = Deck()
for i in range(5):
    hands[0].append(deck.draw())
    hands[1].append(deck.draw())
    for i, hand in enumerate(hands):
        print("Player {}: {}".format(i+1, hand_string(hand)))
        
    # wins = sum([run_simulation(hands) for i in range(N)])
    if __name__ == '__main__':
        wins = sum(Parallel(-1)(delayed(run_simulation)(hands) for i in range(N)))
        
    equity = int(wins/N*100)
    while True:
        try:
            estimate = float(input("Estimate Player 1 equity "))
        except:
            print("Errror. Try again.")
        
    error = estimate-equity
    print("Player 1 equity: {}. Error: {}".format(equity, error))